In [4]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

## 데이터 준비
- 모델에 넘겨줄 데이터 (dls):
- X (입력 데이터): 사용자 ID와 영화 ID.
- Y (타겟 데이터): 사용자별 평점 점수.

In [5]:
# path/'u.data' 기본 테이블 경로
# 사용자ID, 영화ID, 점수 등이 있어.
ratings = pd.read_csv(path/'u.data',
                     delimiter='\t',
                     header=None,
                     names=['user','movie','rating','timestamp'])

ratings.head(2)

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742


In [6]:
# 사용자 간 유사성은 어떻게 구할까?
# SF, 액션, 고전을 좋아하는 정도를 수치로 나타내.
user1 = np.array([0.98, 0.9, -0.9])
user2 = np.array([0.9, 0.8, -0.6])

# user1과 user2의 유사성은 접곰으로 구해.
(user1 * user2).sum()

2.1420000000000003

In [7]:
# 영화ID을 제목으로 대체해.
# 제목 경로는 path/'u.item'
movies = pd.read_csv(path/'u.item',
                     delimiter='|',
                     encoding='latin-1',
                     usecols=(0,1),
                     names=['movie','title'],
                     header=None)

movies.head(2)

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)


In [8]:
# 제목 테이블 (movies) 를 기본 테이블 (ratings) 에 결합해.

ratings = ratings.merge(movies)
ratings.head(2)

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)


## 데이터로더
- CallabDataLoaders 의 디폴트 특징
- 1번열로 user, 2번열로 항목(movie), 3번열로 rating을 가져다가 dls를 만들어.
- **2번열을 movie 대신 title로 바꾸려 해.**
- 열 변경은 user_name, item_name, rating_name 매개변수를 바꾸면 돼.

In [9]:
dls = CollabDataLoaders.from_df(ratings,
                                item_name='title',
                                bs=64)
dls.show_batch()
# dls[:, 0] = user
# dls[:, 1] = title

,user,title,rating
0,721,Kicked in the Head (1997),3
1,345,Speed (1994),4
2,62,Star Wars (1977),5
3,249,My Best Friend's Wedding (1997),3
4,303,"Rock, The (1996)",3
5,716,"Sound of Music, The (1965)",5
6,399,"Lion King, The (1994)",3
7,373,Made in America (1993),3
8,291,"Abyss, The (1989)",4
9,60,Breakfast at Tiffany's (1961),4


## 모델이 수행할 작업
- **임베딩**: 사용자 ID와 영화 ID를 수치(임베딩 벡터)로 변환해.
- **점곱 계산**: 사용자의 임베딩 벡터와 영화의 임베딩 벡터를 요소별로 곱하고, 그 결과를 합산하여 사용자별 평점 **예측** 점수를 계산해.
- **손실 최소화**: 평점 **예측** 점수와 실제 점수 간의 차이(손실)를 줄이기 위해 모델을 학습해.

**임베딩**
- 범주형(ID등)을 수치형으로 바꾸는 것.
- 사용자수를 100, 잠재요소개수를 5라고 하자.

1. Embedding 계층 (user_fatcors) 초기화 : (100*5) 모양.
2. 인덱싱(Indexing) : 사용자ID 넘기기.
- Embedding 내부엔 임베딩 테이블이 있어.
- 이 테이블에는 ID 열과 벡터 열이 저장돼.
- 입력으로 제공된 사용자 ID는 인덱스로 사용돼. <br>
| ID | 벡터          |
|----|-------------|
| 0  | [x1, x2, x3] |
| 1  | [y1, y2, y3] |
| 2  | [z1, z2, z3] |

**점곱**
- user_factors (100*5) 와 movie_factors (100*5) 를 요소별로 곱하고 합산.
- 이 값이 사용자별 평점 예측 점수야. (100) 모양.
- *NumPy의 np.dot() 함수와 달리, 점곱은 동일한 위치에 있는 요소 끼리 곱해.

In [10]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_fatcors = 5

torch.randn(n_users, n_fatcors) # 임의로 만든 n_users*5 행렬.

tensor([[ 0.1531,  0.6339, -0.2521,  0.1538,  1.0126],
        [ 0.4612,  0.2719, -0.6853,  0.2839, -0.0030],
        [ 0.2353,  0.2725,  0.6569, -0.2560,  0.3473],
        ...,
        [-0.5727, -1.0461,  0.5114,  1.6148,  0.1742],
        [ 0.0486,  0.8721,  0.6095,  0.7302, -0.9667],
        [ 0.6913,  1.2748,  1.5968, -0.6550, -1.1866]])

In [11]:
class DotProduct(Module):
    # Embedding 계층 초기화
    def __init__(self, n_users, n_movies, n_factors):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
    
    # 사용자 및 영화 ID 넘기기
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        return (users*movies).sum(dim=1) # 점곱 계산

In [12]:
model = DotProduct(n_users, n_movies, 50)

learn = Learner(dls, model, loss_func=MSELossFlat())

In [13]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.366971,1.274634,00:08
1,1.076361,1.111905,00:08
2,0.976048,1.003460,00:07
3,0.862713,0.904968,00:08
4,0.813333,0.888215,00:08


## 모델 개선: 예측 범위를 0~5가 되도록 강제
- sigmoid_range 함수로 구현

In [14]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.y_range = y_range
        
    # 예측 범위를 0~5가 되도록 강제
    # sigmoid_range함수로 구현
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        return sigmoid_range((users*movies).sum(dim=1), *self.y_range) # unpack

model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.980548,0.991887,00:08
1,0.869839,0.899969,00:08
2,0.681890,0.874633,00:08
3,0.467136,0.875479,00:08
4,0.356470,0.880198,00:08


## 모델 개선: 편향 추가

In [15]:
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias = Embedding(n_users, 1) # 사용자 편향
        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_bias = Embedding(n_movies, 1) # 영화 편향
        self.y_range = y_range
        
    
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        res = (users*movies).sum(dim=1, keepdim=True) # 점곱 계산
        res += self.user_bias(x[:, 0]) + self.movie_bias(x[:, 1]) # 편향 더하기
        return sigmoid_range(res, *self.y_range)

model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.918193,0.932469,00:09
1,0.822338,0.859406,00:08
2,0.604462,0.862971,00:09
3,0.397753,0.888419,00:09
4,0.288703,0.895641,00:09


## 모델 개선: 가중치 감소
- 모델의 가중치를 너무 크게 되지 않도록 제한하는 방법.
- 가중치가 커지면 모델이 훈련 데이터에 과적합(overfitting)될 위험이 있음.
- wd: weight decay 
- 0.1: 각 가중치를 얼마나 줄일 지 나타내는 값

In [16]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.943053,0.947362,00:09
1,0.851943,0.871780,00:09
2,0.737073,0.827995,00:09
3,0.588582,0.817257,00:08
4,0.481843,0.816647,00:08


## 모델 직접 만들기
- Embedding 클래스를 사용하지 않고 만들 것.
- 이 두 가지가 들어가도록 클래스를 만들면 돼.
1. 임베딩하기 <br>
    우선 텐서를 초기화하고 사용자ID를 넘겨 인덱싱 해.
2. 점곱 계산

In [18]:
class T(Module):
    def __init__(self):
        self.a = torch.ones(3)

# 텐서를 초기화한다고 해서 바로 parameters메서드가 반환되지 않아.
# nn.Parameter 클래스로 래핑해야 해.
L(T().parameters())

(#0) []

In [19]:
class T(Module):
    def __init__(self):
        self.a = nn.Parameter(torch.ones(3))
        
L(T().parameters())  

(#1) [Parameter containing:
tensor([1., 1., 1.], requires_grad=True)]

In [122]:
def create_params(size):
    return nn.Parameter(torch.zeros(*size).normal_(0, 0.01))

# normal() : 평균 0, 표준편차 0.01 의 분포를 따르는 텐서로 바꿔.
# _ 접미사 : 이 연산이 원본 텐서를 변경한다는 것을 나타내.

create_params((3,3)) # size는 튜플이나 리스트.

Parameter containing:
tensor([[ 0.0048, -0.0039,  0.0036],
        [-0.0053,  0.0026,  0.0190],
        [-0.0113,  0.0128,  0.0038]], requires_grad=True)

In [123]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
        self.user_factors = create_params((n_users, n_factors))
        self.user_bias = create_params((n_users, 1))
        self.movie_factors = create_params((n_movies, n_factors))
        self.movie_bias = create_params((n_movies, 1))
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors[x[:, 0]] # 인덱싱
        movies = self.movie_factors[x[:, 1]]
        res = (users*movies).sum(dim=1, keepdim=True)
        res += self.user_bias[x[:, 0]] + self.movie_bias[x[:, 1]]
        return sigmoid_range(res, *self.y_range)
    
    
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(3, wd=0.1)

epoch,train_loss,valid_loss,time
0,1.071484,1.049650,00:09
1,0.895926,0.929110,00:09
2,0.887334,0.920864,00:09


**텐서의 인덱싱 (create_params로 생성된 users):**

- **대괄호[]** 를 사용해.
- 리스트 형태의 인덱스, user_id = [2, 1, 0]를 사용하여 여러 행을 한 번에 인덱싱할 수 있어.
- users[user_id]

**Embedding 계층의 인덱싱:**
- **괄호()** 를 사용하고, 인덱스로 **텐서**를 받아.
- users(user_id)

In [120]:
users = create_params((3,3))
user_id = [2,1,0] # 인덱스 리스트
users[user_id]

users[2] # 2번 인덱스 조회

tensor([-0.0081, -0.0053, -0.0060], grad_fn=<SelectBackward0>)

In [121]:
users = nn.Embedding(3,3)
user_id = torch.tensor([2,1,0]) # 인덱스 텐서
users(user_id)

users(torch.tensor([2])) # 2번 인덱스 조회

tensor([[-1.7624,  1.0910,  0.1633]], grad_fn=<EmbeddingBackward0>)